In [1]:
import numpy as np
import pymongo
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error, ndcg_score
import pandas as pd
from tqdm import tqdm
import time
import mlflow
from final.model_processing import run_model, partition

In [2]:
# Define the model hyperparameters
X = 18 # n_components
Y = 150 # max_iter
params = {
    "n_components": X,
    "max_iter": Y,
}
TAG = 'no filtres'

In [3]:
#Define the thresholds under which we drop the movies
MOVIES_SEUIL = 1 #en dessous de ce nombre de note on élimine le movie du dataset
USERS_SEUIL = 1 #en dessous de ce nombre de note on élimine le user du dataset

### Connexion à la db

In [4]:
client = pymongo.MongoClient('localhost:27017')

db = client['Movielens2']

movies = db['movies']
users = db['users']

### Construire une liste à plat: user / movieid / rating / timestamp

In [5]:
# Extraire les données de la collection
data = list(users.find())

# Créer une liste vide pour chaque colonne du DataFrame
user_list = []
movieid_list = []
rating_list = []
timestamp_list = []

# Parcourir les données et extraire les informations nécessaires
for entry in data:
    user_id = entry['_id']
    for movie in entry['movies']:
        user_list.append(user_id)
        movieid_list.append(movie['movieid'])
        rating_list.append(movie['rating'])
        timestamp_list.append(movie['timestamp'])

# Créer le DataFrame
df = pd.DataFrame({
    'user_id': user_list,
    'movie_id': movieid_list,
    'rating': rating_list,
    'timestamp': timestamp_list
})

# Afficher les premières lignes du DataFrame
print(df.head())

   user_id  movie_id  rating  timestamp
0     6040       573       4  956704056
1     6040       589       4  956704996
2     6040         1       3  957717358
3     6040      2068       4  997453982
4     6040       592       2  956716016


In [6]:
#Number of ratings per user
cusers_counts = df['user_id'].value_counts()
print('Nombre de ratings par utilisateur')
print(cusers_counts.describe())

#Note moyenne par film
mean_ratings = df.groupby('movie_id').rating.mean()
print("note moyenne par film")
print(mean_ratings.describe())
#Note moyenne par utilisateur
mean_ratings = df.groupby('user_id').rating.mean()
print("note moyenne par utilisateur")
print(mean_ratings.describe())

Nombre de ratings par utilisateur
count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: count, dtype: float64
note moyenne par film
count    3706.000000
mean        3.238892
std         0.672925
min         1.000000
25%         2.822705
50%         3.331546
75%         3.740741
max         5.000000
Name: rating, dtype: float64
note moyenne par utilisateur
count    6040.000000
mean        3.702705
std         0.429622
min         1.015385
25%         3.444444
50%         3.735294
75%         4.000000
max         4.962963
Name: rating, dtype: float64


In [7]:
#Number of ratings per movie
movies_counts = df['movie_id'].value_counts()
print('Nombre de ratings par film')
movies_counts.describe()

Nombre de ratings par film


count    3706.000000
mean      269.889099
std       384.047838
min         1.000000
25%        33.000000
50%       123.500000
75%       350.000000
max      3428.000000
Name: count, dtype: float64

In [8]:
#Drop movies with less than X ratings
df = df[df['movie_id'].isin(movies_counts[movies_counts > MOVIES_SEUIL].index)]

In [9]:
#Drop users with less than X ratings
df = df[df['user_id'].isin(cusers_counts[cusers_counts > USERS_SEUIL].index)]

#Garder que les utilisateurs ayant une moyenne de rating supérieure à 1.5 et inférieure à 4.5
df = df.groupby('user_id').filter(lambda x : x['rating'].mean() > 1.5 and x['rating'].mean() < 4.5)

print('Nouvelle taille de df :', len(df))

Nouvelle taille de df : 992028


### Splitter les données en test et train

In [10]:
df_train, df_test, train_mini, test_mini = partition(df)
df_train.shape, df_test.shape

((793622, 4), (198406, 4))

### un peu de détail sur df_train et df_test

In [11]:
# Créer un ensemble d'utilisateurs pour chaque DataFrame
users_train = set(df_train['user_id'])
users_test = set(df_test['user_id'])

# Trouver les utilisateurs communs
users_common = users_train.intersection(users_test)

# Convertir l'ensemble en liste si nécessaire
users_common_list = list(users_common)
print("Nombre d'utilisateurs communs aux deux DataFrames :", len(users_common))

# Calculer les utilisateurs dans le train mais pas dans le test
users_train_not_in_test = users_train - users_test
num_users_train_not_in_test = len(users_train_not_in_test)

# Calculer les utilisateurs dans le test mais pas dans le train
users_test_not_in_train = users_test - users_train
num_users_test_not_in_train = len(users_test_not_in_train)

print("Nombre d'utilisateurs dans le train mais pas dans le test :", num_users_train_not_in_test)
print("Nombre d'utilisateurs dans le test mais pas dans le train :", num_users_test_not_in_train)

Nombre d'utilisateurs communs aux deux DataFrames : 5932
Nombre d'utilisateurs dans le train mais pas dans le test : 5
Nombre d'utilisateurs dans le test mais pas dans le train : 0


In [12]:
# Créer un ensemble de movies pour chaque DataFrame
movies_train = set(df_train['movie_id'])
movies_test = set(df_test['movie_id'])

# Trouver les movies communs
movies_common = movies_train.intersection(movies_test)

# Convertir l'ensemble en liste si nécessaire
movies_common_list = list(movies_common)
print("Nombre de film communs aux deux DataFrames :", len(movies_common))

# Calculer les utilisateurs dans le train mais pas dans le test
movies_train_not_in_test = movies_train - movies_test
num_movies_train_not_in_test = len(movies_train_not_in_test)

# Calculer les utilisateurs dans le test mais pas dans le train
movies_train_not_in_train = movies_test - movies_train
num_movies_test_not_in_train = len(movies_train_not_in_train)

print("Nombre de movies dans le train mais pas dans le test :", num_movies_train_not_in_test)
print("Nombre de movies  dans le test mais pas dans le train :", num_movies_test_not_in_train)

Nombre de film communs aux deux DataFrames : 3424
Nombre de movies dans le train mais pas dans le test : 165
Nombre de movies  dans le test mais pas dans le train : 3


### Clean des données (user et movie) de TEST qui ne sont pas inclus dans train

In [13]:
# Filtrer les lignes de df_test pour lesquelles 'user' est dans users_common_list
df_test_filtered_user = df_test[df_test['user_id'].isin(users_common_list)]

# Filtrer les lignes de df_test pour lesquelles 'movies' est dans users_common_list
df_test_filtered = df_test_filtered_user[df_test_filtered_user['movie_id'].isin(movies_common_list)]

# Afficher le DataFrame filtré
print(df_test_filtered)

        user_id  movie_id  rating  timestamp
25334      5880      2797       2  959183085
816053     1146      1480       2  974938637
140254     5171      3654       4  961889611
573365     2592      1947       3  973827611
623621     2197      1196       5  974605842
...         ...       ...     ...        ...
738959     1597      2409       2  989847326
498550     3081      2793       4  969801483
240291     4514      1979       2  965959372
413024     3586      1939       5  966691799
773377     1376      2716       4  974767719

[198400 rows x 4 columns]


### Trouver l'utilisateur commun qui a vu le + de film dans le df_test

In [14]:
# Compter le nombre de films vus par chaque utilisateur dans df_test
films_vus_par_utilisateur_test = df_test.groupby('user_id')['movie_id'].count()

# Compter le nombre de films vus par chaque utilisateur dans df_train
films_vus_par_utilisateur_train = df_train.groupby('user_id')['movie_id'].count()

# Filtrer les utilisateurs communs
users_common_list_combined = list(set(films_vus_par_utilisateur_test.index) & set(films_vus_par_utilisateur_train.index))

# Filtrer les films vus par les utilisateurs communs dans df_test
films_vus_par_utilisateur_common_test = films_vus_par_utilisateur_test[users_common_list_combined]

# Filtrer les films vus par les utilisateurs communs dans df_train
films_vus_par_utilisateur_common_train = films_vus_par_utilisateur_train[users_common_list_combined]

# Calculer le nombre total de films vus par chaque utilisateur dans les deux ensembles de données
films_vus_par_utilisateur_combined = films_vus_par_utilisateur_common_test + films_vus_par_utilisateur_common_train

# Trouver l'utilisateur avec le plus grand nombre de films vus
utilisateur_max_films_vus_combined = films_vus_par_utilisateur_combined.idxmax()

# Afficher l'utilisateur avec le plus grand nombre de films vus
print("L'utilisateur avec le plus grand nombre de films vus à la fois dans df_test et df_train est :", utilisateur_max_films_vus_combined)


L'utilisateur avec le plus grand nombre de films vus à la fois dans df_test et df_train est : 4169


### Entrainer le modele NMF et sortir une matrice complétée

In [15]:
options = {
    'n_components': X,
    'max_iter': Y,
    'normalize': {
        'should': True,
        'min': 1,
        'max': 5}
}

# model, predict_matrix = run_model(train_df, options)
start_time = time.time()

model, df_depivoted, pred_matrix= run_model(df_train, options)


# Fin du chronomètre
end_time = time.time()

# Calcul du temps écoulé en secondes
training_time = end_time - start_time

print("Temps d'entraînement:", training_time, "secondes")
model, df_depivoted.shape, pred_matrix

/home/pauline/miniconda3/envs/env_br08/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 150 reached. Increase it to improve convergence.
  warnings.warn(


Temps d'entraînement: 6.838233947753906 secondes


(NMF(max_iter=150, n_components=18),
 (21307893, 3),
 movie_id      1         2         3         4         5         6     \
 user_id                                                                
 1         4.500933  2.153153  1.068352  1.012719  1.077313  1.089707   
 2         2.442313  1.409540  1.161869  1.098755  1.076142  1.938462   
 3         4.406630  1.377603  1.018504  1.000000  1.010171  1.586816   
 4         1.334737  1.001312  1.001195  1.005316  1.000000  1.372011   
 5         1.642486  1.027321  1.000699  1.112665  1.005235  2.289019   
 ...            ...       ...       ...       ...       ...       ...   
 6036      3.181588  1.965668  1.192056  1.362644  1.160362  2.193584   
 6037      2.532465  1.068475  1.000870  1.009647  1.007358  1.132575   
 6038      4.207695  1.410286  1.141741  1.061722  1.073444  1.216393   
 6039      3.843330  1.795600  1.170931  1.072070  1.105743  1.032469   
 6040      2.205729  1.045941  1.005305  1.115760  1.000000  1.373775  

### Obtenir une matrice de rating réél

In [16]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

### Calculer les métriques

In [17]:
# Calculate metrics
df_compare_seen_movie_train = pd.merge(df_depivoted, df_train, how='inner', on=['user_id', 'movie_id'])
df_compare_seen_movie_test = pd.merge(df_depivoted, df_test, how='inner', on=['user_id', 'movie_id'])

#MSE et Root MSE

mse = mean_squared_error(df_compare_seen_movie_train['rating'], df_compare_seen_movie_train['predict'])
root_mse = np.sqrt(mse)

mse_test = mean_squared_error(df_compare_seen_movie_test['rating'], df_compare_seen_movie_test['predict'])
rmse_test = np.sqrt(mse_test)

#Correlation de Pierson

correlations_train = []
for u in tqdm(df_compare_seen_movie_train['user_id'].unique()):
    section_user = df_compare_seen_movie_train[df_compare_seen_movie_train.user_id == u]
    correlation = section_user['predict'].corr(section_user['rating'], method='pearson')
    correlations_train.append(correlation)
moy_correlation = sum(correlations_train) / len(correlations_train)

correlations_test = []
for u in tqdm(df_compare_seen_movie_test['user_id'].unique()):
    section_user = df_compare_seen_movie_test[df_compare_seen_movie_test.user_id == u]
    if section_user['predict'].var() != 0 and section_user['rating'].var() != 0:
        correlation = section_user['predict'].corr(section_user['rating'], method='pearson')
        correlations_test.append(correlation)
    else:
        print(f"l'utilisateur {u} a mis des notes égales")
moy_correlation_test = sum(correlations_test) / len(correlations_test)

#Compute Normalized Discounted Cumulative Gain

ndcg_score_train = []
for u in tqdm(df_compare_seen_movie_train['user_id'].unique()):
    section_user = df_compare_seen_movie_train[df_compare_seen_movie_train.user_id == u]
    score = ndcg_score([section_user['rating'].values], [section_user['predict'].values])
    ndcg_score_train.append(score)
moy_ndcg_score_train = sum(ndcg_score_train) / len(ndcg_score_train)

ndcg_score_test = []
for u in tqdm(df_compare_seen_movie_test['user_id'].unique()):
    section_user = df_compare_seen_movie_test[df_compare_seen_movie_test.user_id == u]
    score = ndcg_score([section_user['rating'].values], [section_user['predict'].values])
    ndcg_score_test.append(score)
moy_ndcg_score_test = sum(ndcg_score_test) / len(ndcg_score_test)

print('mse train:', mse)
print('mse test:', mse_test)
print('root_mse train:', root_mse)
print('root_mse test:', rmse_test)
print('moy_correlation train:', moy_correlation)
print('moy_correlation test:', moy_correlation_test)
print('ndcg_score train:', moy_ndcg_score_train)
print('ndcg_score test:', moy_ndcg_score_test)

  3%|██▍                                                                            | 182/5932 [00:00<00:06, 930.65it/s]

l'utilisateur 21 a mis des notes égales
l'utilisateur 152 a mis des notes égales


  7%|█████▏                                                                         | 388/5932 [00:00<00:05, 971.81it/s]

l'utilisateur 254 a mis des notes égales
l'utilisateur 421 a mis des notes égales


  8%|██████▍                                                                        | 486/5932 [00:00<00:06, 897.46it/s]/home/pauline/miniconda3/envs/env_br08/lib/python3.11/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/pauline/miniconda3/envs/env_br08/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/pauline/miniconda3/envs/env_br08/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
 10%|███████▊                                                                       | 584/5932 [00:00<00:05, 922.86it/s]

...

l'utilisateur 5579 a mis des notes égales
l'utilisateur 5666 a mis des notes égales
l'utilisateur 5784 a mis des notes égales


 99%|█████████████████████████████████████████████████████████████████████████████ | 5861/5932 [00:06<00:00, 855.46it/s]

l'utilisateur 5873 a mis des notes égales
l'utilisateur 5898 a mis des notes égales
l'utilisateur 5944 a mis des notes égales
l'utilisateur 5999 a mis des notes égales


  9%|███████▍                                                                       | 554/5932 [00:00<00:05, 912.41it/s]


ValueError: Only ('multilabel-indicator', 'continuous-multioutput', 'multiclass-multioutput') formats are supported. Got binary instead

### + Logger et enregistrer les modeles sur MLFlow

In [ ]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Movielens")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("mse_train", mse)
    mlflow.log_metric("mse_test", mse_test)
    mlflow.log_metric("root_mse_train", root_mse )
    mlflow.log_metric("root_mse_test", rmse_test )
    mlflow.log_metric("time_to_fit", training_time)
    mlflow.log_metric("correlation-train", moy_correlation)
    mlflow.log_metric("correlation-test", moy_correlation_test)
    mlflow.log_metric("ndcg_score train", moy_ndcg_score_train)
    mlflow.log_metric("ndcg_score test", moy_ndcg_score_test)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", TAG)

    # Infer the model signature
    #signature = infer_signature(df_train, pred_matrix_train)

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=NMF,
        artifact_path="nmf_model",
        signature=False,
        input_example=df_train,
        registered_model_name=f"nmf {X} components ",
    )

In [ ]:
# # Créer des tableaux NumPy à partir des DataFrames
# array_to_filter = df_depivoted.to_numpy()
# array_reference = df_train.to_numpy()[:, :2]  # Utiliser uniquement les colonnes 'user' et 'movieid' pour la référence
# print(array_reference)
# print(array_to_filter)

In [ ]:
# # Trouver les index des lignes à supprimer
# rows_to_remove = np.isin(array_to_filter[:, :2], array_reference, invert=True).any(axis=1)

# # Supprimer les lignes du tableau à filtrer
# filtered_array = array_to_filter[rows_to_remove]

# # Convertir le tableau filtré en DataFrame si nécessaire
# # df_predict = pd.DataFrame(filtered_array, columns=['user', 'movieid', 'rating'])

In [ ]:
# df_predict_movie_not_seen_4169 = df_predict_movie_not_seen.loc[df_predict_movie_not_seen['user']== 4169].sort_values(by = ['predict'], ascending=False)
# df_predict_movie_not_seen_4169

In [ ]:
# df_compare_not_seen_movie_4169 = pd.merge(df_predict_movie_not_seen_4169, df_test_filtered, how='inner', on=['user', 'movieid'])
# df_compare_not_seen_movie_4169

In [ ]:
# df_compare_not_seen_movie_4169.sort_values(by = ['rating'], ascending=False)